In [0]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

from tensorflow import keras
import os
import re

In [42]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [0]:
import shutil

drive_path = ""
filenames = ["ADelete_entries.json", "DASH_entries.json", "T_entries.json"]

for filename in filenames:
  shutil.copy2('/content/gdrive/My Drive/'+drive_path+filename,'.')

In [44]:
os.listdir(".")

['.config',
 'DASH_entries.json',
 'T_entries.json',
 'ADelete_entries.json',
 'realec_110319_2315.tar.gz',
 'All_Entries (1).json',
 'data',
 'All_Entries.json',
 'gdrive',
 'adc.json',
 'sample_data']

In [0]:
for jsonname in filenames:
  exec(jsonname[:-5]+" = pd.read_json(jsonname).reset_index(drop=True)")

In [46]:
T_entries[:10]

,path,line
0,./data/exam/exam2017_3/DOv_7_1.ann,T1\tPrepositions 60 64\tfrom
1,./data/exam/exam2017_3/DOv_7_1.ann,T2\tRedundant_comp 79 85\tgroups
2,./data/exam/exam2017_3/DOv_7_1.ann,T11\tAgreement_errors 314 321\tbecomes
3,./data/exam/exam2017_3/DOv_4_1.ann,T16\tPossessive 777 786\tLinked In
4,./data/exam/best_works/36_1.ann,T12\tTense_choice 698 707\tcomprises
5,./data/exam/exam2017/DOv_26_1.ann,T13\tlex_item_choice 1016 1019\tany
6,./data/old IELTS/IELTS2016/EEm_123_1.ann,T7\tArticles 706 715\tgoverment
7,./data/old IELTS/IELTS2016/EEm_123_1.ann,T8\tPrepositions 814 816\tin
8,./data/old IELTS/IELTS2016/EEm_123_1.ann,T9\tSpelling 877 886\tgoverment
9,./data/old IELTS/IELTS2016/EEm_123_1.ann,T10\tSpelling 907 916\tsuccesful


In [47]:
DASH_entries[:10]

,path,line
0,./data/exam/exam2017_3/DOv_7_1.ann,#1\tAnnotatorNotes T1\tof
1,./data/exam/exam2017_3/DOv_7_1.ann,#2\tAnnotatorNotes T3\tages
2,./data/exam/exam2017_3/DOv_7_1.ann,#11\tAnnotatorNotes T12\tFacebook and Instagra...
3,./data/exam/exam2017_3/DOv_4_1.ann,#17\tAnnotatorNotes T19\ttarget audiences
4,./data/exam/best_works/10_2.ann,"#16\tAnnotatorNotes T16\t, but"
5,./data/exam/best_works/23_2.ann,#244\tAnnotatorNotes T245\tlemma = 'cause'
6,./data/exam/exam2017/DPe_62_2.ann,#229\tAnnotatorNotes T229\tlemma = 'it'
7,./data/exam/exam2014/2012-2014_2/esl_00450.ann,#334\tAnnotatorNotes T341\tlemma = 'Earth'
8,./data/exam/exam2014/2012-2014_2/esl_00450.ann,#335\tAnnotatorNotes T342\tlemma = 'and'
9,./data/exam/exam2014/2012-2014_2/esl_00450.ann,#336\tAnnotatorNotes T343\tlemma = 'start'


In [48]:
ADelete_entries[:10]

,path,line
0,./data/exam/exam2017_3/DOv_7_1.ann,A1\tDelete T2
1,./data/exam/exam2017_3/DOv_4_1.ann,A1\tDelete T9
2,./data/exam/best_works/25_2.ann,A2\tDelete T15
3,./data/exam/best_works/7_2.ann,A2\tDelete T22
4,./data/exam/exam2017/ABl_25_2.ann,A2\tDelete T25
5,./data/exam/exam2017/ABl_25_2.ann,A3\tDelete T32
6,./data/exam/exam2017/ABl_25_2.ann,A4\tDelete T42
7,./data/exam/exam2017/ABl_25_2.ann,A5\tDelete T68
8,./data/exam/exam2017/ABl_25_2.ann,A6\tDelete T84
9,./data/exam/exam2017/OBy_168_2.ann,A1\tDelete T20


Let's rearrange this all into one single database!

In [0]:
linesplit = list(T_entries.line.apply(lambda x: [x.split('\t')[0]] + x.split('\t')[1].split(' ') + [x.split('\t')[2]] ))

In [0]:
All_Entries = pd.DataFrame({
  'path': list(T_entries.path),
  'id': [x[0] for x in linesplit],
  'type': [x[1] for x in linesplit],
  'begin': [x[2] for x in linesplit],
  'end': [x[3] for x in linesplit],
  'substring': [x[4] for x in linesplit],
  'correction': ['' for _ in linesplit],
  'delete': [False for _ in linesplit]
})

In [51]:
All_Entries.head()

,path,id,type,begin,end,substring,correction,delete
0,./data/exam/exam2017_3/DOv_7_1.ann,T1,Prepositions,60,64,from,,False
1,./data/exam/exam2017_3/DOv_7_1.ann,T2,Redundant_comp,79,85,groups,,False
2,./data/exam/exam2017_3/DOv_7_1.ann,T11,Agreement_errors,314,321,becomes,,False
3,./data/exam/exam2017_3/DOv_4_1.ann,T16,Possessive,777,786,Linked In,,False
4,./data/exam/best_works/36_1.ann,T12,Tense_choice,698,707,comprises,,False


In [52]:
DASHZip = zip(list(DASH_entries.path), list(DASH_entries.line))
DASHList = []
for e in DASHZip:
  DASHList.append([e[0], e[1].split('\t')[1].split(' ')[1], e[1].split('\t')[2]])
DASHList[:10]

[['./data/exam/exam2017_3/DOv_7_1.ann', 'T1', 'of'],
 ['./data/exam/exam2017_3/DOv_7_1.ann', 'T3', 'ages'],
 ['./data/exam/exam2017_3/DOv_7_1.ann',
  'T12',
  'Facebook and Instagram become among them'],
 ['./data/exam/exam2017_3/DOv_4_1.ann', 'T19', 'target audiences'],
 ['./data/exam/best_works/10_2.ann', 'T16', ', but'],
 ['./data/exam/best_works/23_2.ann', 'T245', "lemma = 'cause'"],
 ['./data/exam/exam2017/DPe_62_2.ann', 'T229', "lemma = 'it'"],
 ['./data/exam/exam2014/2012-2014_2/esl_00450.ann', 'T341', "lemma = 'Earth'"],
 ['./data/exam/exam2014/2012-2014_2/esl_00450.ann', 'T342', "lemma = 'and'"],
 ['./data/exam/exam2014/2012-2014_2/esl_00450.ann', 'T343', "lemma = 'start'"]]

In [0]:
DASHDict = {e[0]+e[1]: e[2] for e in DASHList}

In [54]:
%%time

for i, row in All_Entries.iterrows():
  key = row[0]+row[1]
  Correction = ""
  if key in DASHDict:
    Correction = DASHDict[key]
  All_Entries.at[i,'correction'] = Correction

CPU times: user 17.1 s, sys: 37.7 ms, total: 17.1 s
Wall time: 17.1 s


In [55]:
All_Entries.head()

,path,id,type,begin,end,substring,correction,delete
0,./data/exam/exam2017_3/DOv_7_1.ann,T1,Prepositions,60,64,from,of,False
1,./data/exam/exam2017_3/DOv_7_1.ann,T2,Redundant_comp,79,85,groups,,False
2,./data/exam/exam2017_3/DOv_7_1.ann,T11,Agreement_errors,314,321,becomes,become,False
3,./data/exam/exam2017_3/DOv_4_1.ann,T16,Possessive,777,786,Linked In,LinkedIn's,False
4,./data/exam/best_works/36_1.ann,T12,Tense_choice,698,707,comprises,was,False


In [0]:
ADZip = zip(list(ADelete_entries.path), list(ADelete_entries.line))
ADSet = set()
for e in ADZip:
  ADSet.add(e[0]+e[1].split('\t')[1].split(' ')[1])

In [57]:
%%time

for i, row in All_Entries.iterrows():
  key = row[0]+row[1]
  Delete = False
  if key in ADSet:
    All_Entries.at[i,'delete'] = True

CPU times: user 15.1 s, sys: 1.9 ms, total: 15.1 s
Wall time: 15.1 s


In [58]:
All_Entries.head()

,path,id,type,begin,end,substring,correction,delete
0,./data/exam/exam2017_3/DOv_7_1.ann,T1,Prepositions,60,64,from,of,False
1,./data/exam/exam2017_3/DOv_7_1.ann,T2,Redundant_comp,79,85,groups,,True
2,./data/exam/exam2017_3/DOv_7_1.ann,T11,Agreement_errors,314,321,becomes,become,False
3,./data/exam/exam2017_3/DOv_4_1.ann,T16,Possessive,777,786,Linked In,LinkedIn's,False
4,./data/exam/best_works/36_1.ann,T12,Tense_choice,698,707,comprises,was,False


In [59]:
All_Entries.loc[(All_Entries["correction"] == "") & (All_Entries["delete"] == False)]

,path,id,type,begin,end,substring,correction,delete
6,./data/old IELTS/IELTS2016/EEm_123_1.ann,T7,Articles,706,715,goverment,,False
8,./data/old IELTS/IELTS2016/EEm_123_1.ann,T9,Spelling,877,886,goverment,,False
10,./data/old IELTS/IELTS2016/EEm_123_1.ann,T11,Articles,877,886,goverment,,False
12,./data/old IELTS/IELTS2016/EEm_123_1.ann,T13,Numerals,533,541,millions,,False
13,./data/old IELTS/IELTS2016/EEm_123_1.ann,T14,Numerals,550,558,millions,,False
14,./data/old IELTS/IELTS2016/EEm_123_1.ann,T15,Numerals,580,588,millions,,False
15,./data/old IELTS/IELTS2016/EEm_123_1.ann,T16,Numerals,596,604,millions,,False
17,./data/old IELTS/IELTS2016/EEm_123_1.ann,T17,Numerals,630,638,millions,,False
18,./data/old IELTS/IELTS2016/EEm_123_1.ann,T18,Numerals,647,655,millions,,False
108,./data/old IELTS/IELTS2016/DZu_19_2.ann,T10,Inappropriate_register,864,879,"I am 100% sure,",,False


# Adding contexts

Let's get our data here

In [60]:
import shutil

drive_path = "Clean REALEC dumps/" #@param {type:"string"}
filename = "realec_110319_2315.tar.gz" #@param {type:"string"}


shutil.copy2('/content/gdrive/My Drive/'+drive_path+filename,'.')

'./realec_110319_2315.tar.gz'

In [0]:
import tarfile

tar = tarfile.open(filename)
tar.extractall()

Time to create another monster: a structure to contain all the texts we're interested in

In [62]:
%%time

Texts = list(set([x[:-4]+'.txt' for x in list(All_Entries["path"])]))
Text_Dict = {path: open(path, 'r', encoding='utf-8-sig').read() for path in Texts}

CPU times: user 212 ms, sys: 81.1 ms, total: 293 ms
Wall time: 293 ms


In [63]:
print(len(Text_Dict))

5554


Now let's generate entries for our given tokens

In [64]:
import nltk

nltk.download("punkt")
nltk.download("perluniprops")

from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

from nltk.tokenize.moses import MosesDetokenizer
detokenizer = MosesDetokenizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!


We have to change our substrings first to a character that never appears in the corpus to avoid further confusion. Let's check some things...

In [65]:
chr(8)

'\x08'

In [66]:
contain = []

for el in Text_Dict:
  if chr(8) in Text_Dict[el]:
    contain.append(Text_Dict[el])

print(len(contain))

0


Perfect.

In [0]:
def maskify(instr):
  replist = tknzr.tokenize(instr)
  for replacement in replist:
    instr = instr.replace(replacement, chr(8), 1)
  instr = instr.replace(chr(8)+chr(8), chr(8)+' '+chr(8))
  spacechars = chr(9)+chr(10)+chr(160)+" "
  intext = re.sub(r'[^'+spacechars+']'+chr(8), ' '+chr(8), instr)
  intext = re.sub(chr(8)+r'[^'+spacechars+']', chr(8)+' ', instr)
  if not re.search(chr(8)+r'['+spacechars+']', intext):
    intext = intext.replace(chr(8), chr(8)+' ')
  instr = instr.replace(chr(8), "[MASK]")
  return instr

Yes, this means that we're masking punctuation. So if we dropped a number of sentence delimeters here, we will just concatenate this contexts with adjacent sentences in future

In [68]:
maskify("Although, in my child time I wasn't a child,")

'[MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK]'

In [0]:
All_Entries['context'] = ""

In [0]:
All_Entries.begin = All_Entries.begin.astype(int)
All_Entries.end = All_Entries.end.astype(int)

In [71]:
%%time

for i, row in All_Entries.iterrows():
  fpath = row['path'][:-4]+'.txt'
  start = row['begin']
  end = row['end']
  raw = Text_Dict[fpath]
  substr = raw[start:end]
  raw = raw[:start] + maskify(substr) + raw[end:]
  raw = re.sub(r'\s', ' ', raw)
  raw = re.sub(r'( )+', ' ', raw)
  sentences = nltk.sent_tokenize(raw)
  for k in range(len(sentences)):
    if '[MASK]' in sentences[k]:
      if k < 1:
        All_Entries.at[i,'context'] = ' '.join(sentences[0:k+2])
      else:
        All_Entries.at[i,'context'] = ' '.join(sentences[k-1:k+2])
      break

CPU times: user 1min 55s, sys: 224 ms, total: 1min 56s
Wall time: 1min 56s


In [72]:
All_Entries.head()

,path,id,type,begin,end,substring,correction,delete,context
0,./data/exam/exam2017_3/DOv_7_1.ann,T1,Prepositions,60,64,from,of,False,On the line chart we can see how online adults...
1,./data/exam/exam2017_3/DOv_7_1.ann,T2,Redundant_comp,79,85,groups,,True,On the line chart we can see how online adults...
2,./data/exam/exam2017_3/DOv_7_1.ann,T11,Agreement_errors,314,321,becomes,become,False,We can see that Facebook is the most popular s...
3,./data/exam/exam2017_3/DOv_4_1.ann,T16,Possessive,777,786,Linked In,LinkedIn's,False,"So Instagram and LinkedIn has similar rates, w..."
4,./data/exam/best_works/36_1.ann,T12,Tense_choice,698,707,comprises,was,False,Comparing two the most successful companies it...


Let's deal with these strange cases where `begin` and `end` values differ but we're not given any `substring`. We should just get it manually!

In [0]:
for i, row in All_Entries.iterrows():
  if (row['end'] > row['begin']) and (row['substring'] == ""):
    fpath = row['path'][:-4]+'.txt'
    start = int(row['begin'])
    end = int(row['end'])
    raw = Text_Dict[fpath]
    All_Entries.at[i,'substring'] = raw[start:end]

Now all what is left is to also count the length of substrings in terms of words as measured by `nltk.TweetTokenizer`

In [0]:
def twttknzr_length(instr):
  return len(tknzr.tokenize(instr))

In [0]:
All_Entries['substr_words'] = All_Entries.apply(lambda x: twttknzr_length(x['substring']), axis=1)

In [0]:
All_Entries.loc[All_Entries['type'] == 'lex_part_choice']

## Uploading the results

Wow. That was epic and took a looong time. We'd better save it into a json file to accelerate future load-ups.

In [76]:
%%time

jsonname = "All_Entries.json" #@param {type:"string"}
with open(jsonname, 'w', encoding="utf-8") as outie:
    exec("outie.write("+jsonname[:-5]+".to_json())")

CPU times: user 342 ms, sys: 95 ms, total: 437 ms
Wall time: 438 ms


Whew, great! Let's upload this to Google Drive right now. 

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [78]:
# Create & upload a file.
uploaded = drive.CreateFile({'title': jsonname})
uploaded.SetContentFile(jsonname)
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

W0429 16:20:07.569351 140327652452224 __init__.py:44] file_cache is unavailable when using oauth2client >= 4.0.0
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNo

Uploaded file with ID 1iekC92H3Q1qt0SSPMIgp1HtqstafZaXh
